Tensor RTに変換された学習済みモデルをつかって自動走行します。

In [1]:
import torch
import torchvision

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model = model.cuda().eval().half()

Tensor RT形式のモデルを読み込む。

In [2]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))

<All keys matched successfully>

racecarクラスをインスタンス化する

In [3]:
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

カメラの起動のためにカメラを制御するnvargus-daemonの再起動。

In [4]:
!echo jetson | sudo -S systemctl restart nvargus-daemon

[sudo] password for jetson: 

カメラクラスをインスタンス化する。

In [5]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=40)

最後に、JetRacerを下に置き、下記セルを実行します。

* 車の左右にうまく回らない場合は、`STEERING_GAIN` を小さいくする
* ターンがうまくいかない場合は、`STEERING_GAIN`を大きくする
* 車が左に傾く場合は、`STEERING_BIAS`を値を-0.05ぐらいづつマイナスにする
* 車が右に傾く場合は、`STEERING_BIAS`を値を+0.05ぐらいづつプラスにする

|値|意味|
|:--|:--|
|st_gain|ハンドルの曲がる比率を調整(推論開始ボタンを押したタイミングで反映)|
|st_offset|ハンドルの初期位置を調整(推論開始ボタンを押したタイミングで反映)|

In [18]:
# 車両パラメータを初期化します
car.steering = 0
car.steering_gain = 0.3
car.steering_offset = 0
car.steering_channel = 0
car.throttle = 0
car.throttle_gain = 0.69
car.throttle_offset = 0
car.throttle_channel = 1

In [19]:
import ipywidgets.widgets as widgets
from IPython.display import display
from jetracer.utils import preprocess
import numpy as np
import threading
import traitlets
import time

style = {'description_width': 'initial'}
steering_slider = widgets.FloatSlider(description='steering', style=style, min=-1.0, max=1.0, step=0.01, orientation='horizontal')
steering_gain = widgets.BoundedFloatText(description='steering_gain', style=style ,min=-1.0, max=1.0, step=0.01, value=car.steering_gain)
steering_offset = widgets.BoundedFloatText(description='steering_offset', style=style, min=-1.0, max=1.0, step=0.01, value=car.steering_offset)
throttle_slider = widgets.FloatSlider(description='throttle', style=style, min=-1.0, max=1.0, step=0.01, orientation='vertical')
throttle_gain = widgets.BoundedFloatText(description='throttle_gain', style=style, min=-1.0, max=1.0, step=0.01, value=car.throttle_gain)
throttle_offset = widgets.BoundedFloatText(description='throttle_offset', style=style, min=-1.0, max=1.0, step=0.01, value=car.throttle_offset)

# create a horizontal box container to place the sliders next to eachother
slider_container = widgets.HBox([throttle_slider, steering_slider])
slider_container.layout.align_items='center'
value_container = widgets.VBox([steering_gain, steering_offset, throttle_gain, throttle_offset])

control_container = widgets.HBox([slider_container, value_container])
control_container.layout.align_items='center'

# display the container in this cell's output
display(control_container)


steering_link = traitlets.link((steering_slider, 'value'), (car, 'steering'))
steering_gain_link = traitlets.link((steering_gain, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_offset, 'value'), (car, 'steering_offset'))
throttle_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))
throttle_gain_link = traitlets.link((throttle_gain, 'value'), (car, 'throttle_gain'))
throttle_offset_link = traitlets.link((throttle_offset, 'value'), (car, 'throttle_offset'))


check_button = widgets.Button(description='ハンドルのチェック')
run_button = widgets.Button(description='推論開始')
stop_button = widgets.Button(description='推論停止')
log_widget = widgets.Textarea(description='ログ')
result_widget = widgets.FloatText(description='推論から導いたXの値')

def live():
    global running, count
    log_widget.value = "live"
    count = 0
    while running:
        count = count + 1
        log_widget.value = str(count) + "回目の推論"
        image = camera.read()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0])
        #y = float(output[1])
        result_widget.value = x
        car.steering = x
        #car.throttle = y

def run(c):
    global running, execute_thread, start_time
    log_widget.value = "run"
    running = True
    execute_thread = threading.Thread(target=live)
    execute_thread.start()
    start_time = time.time()

def stop(c):
    global running, execute_thread, start_time, count
    end_time = time.time() - start_time
    fps = count/int(end_time)
    log_widget.value = "FPS: " + str(fps) + "(1秒あたりの推論実行回数)"
    running = False
    execute_thread.stop()
    
def check(c):
    global running, execute_thread, start_time, count
    end_time = time.time() - start_time
    fps = count/int(end_time)
    log_widget.value = "チェック用に推論を停止します。FPS: " + str(fps) + "(1秒あたりの推論実行回数)"
    running = False
    count = 0
    log_widget.value = "car.steering:1"
    car.steering = 1
    time.sleep(1)
    car.steering = -1
    time.sleep(1)
    car.steering = 0
    
run_button.on_click(run)
stop_button.on_click(stop)
check_button.on_click(check)

# create a horizontal box container to place the sliders next to eachother
run_widget = widgets.VBox([
    widgets.HBox([check_button]),
    widgets.HBox([run_button, stop_button]),
    result_widget,
    log_widget
])

# display the container in this cell's output
display(run_widget)